In [4]:
 %%capture
#!pip install wget
#!pip install torchsummaryX
#!pip install wandb

In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [7]:
import pandas as pd
import cv2
import wget
import shutil
from tqdm.auto import tqdm
from joblib import Parallel, delayed
from collections import defaultdict
import random
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import re
import json
import torch
import torch.utils.data as data
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchsummaryX import summary
from torch.optim import lr_scheduler
from IPython.core.display import display, HTML
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from datetime import datetime
import time
from os.path import exists
now = datetime.now()

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
#wget https://github.com/Toloka/WSDMCup2023/raw/main/train.csv

In [ ]:
DATA_DIR = "/home/ngoc/data/WSDM2023/"
TRAIN_IMGS_DIR = DATA_DIR + "train_imgs/"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 4                                        
NUM_EPOCHS = 50                   
EMBEDDIND_SIZE = 1024            
WORD_EMBEDING_SIZE = 300                     
NUMBER_LSTM_LAYER = 1                                            
LEARNING_RATE = 0.00001                                                               
IMAGE_SIZE = 640                                   
LOSS = "RMSLE"
SAVE_IMAGE_FREQ = int(NUM_EPOCHS/5)
LSTM_HIDDEN_SIZE = 1024

In [ ]:
TEST_TRAINING  = True
LOG_WANDB = True

In [ ]:

if LOG_WANDB:
    import wandb
    wandb.init(project="WSDM2023",
               entity="ngocdunghuynh",
                config = {
                    "learning_rate": LEARNING_RATE,
                    "epochs": NUM_EPOCHS,
                    "batch_size": BATCH_SIZE,
                    "device": DEVICE,
                    "loss_function": LOSS,
                })

In [ ]:
train = pd.read_csv(DATA_DIR  + 'train.csv')

In [ ]:
# Download data
try:
    os.mkdir(TRAIN_IMGS_DIR)
    img_paths = Parallel(
    n_jobs=100)(delayed(wget.download)(img_url, out= TRAIN_IMGS_DIR) for img_url in tqdm(train.image)
    )
except:
    print("the training set has been downloaded")

In [ ]:
#Get the image path from URL
train.image  = train.image.apply(lambda x: str(x.split("/")[-1]))
train.image  = train.image.apply(lambda x: TRAIN_IMGS_DIR + str(x))

In [ ]:
#Spit the original training set into 2 dataframes
train_sample = int(len(train)*0.9)
df_train = train[:int(len(train)*0.9)]
df_val = train[int(len(train)*0.9):]

df_train = df_train.reset_index(drop =True)
df_val = df_val.reset_index(drop =True)

if TEST_TRAINING:
    df_train = df_train[:BATCH_SIZE*100]
    df_val = df_val[:BATCH_SIZE*100]

In [ ]:
def tokenize(sentence):

        """Split string lines into lists"""

        SENTENCE_SPLIT_REGEX = re.compile(r'(\W+)')
        tokens = SENTENCE_SPLIT_REGEX.split(sentence.lower())
        tokens = [t.strip() for t in tokens if len(t.strip()) > 0]
        return tokens
        
class Text_Dict:

    """Aim to convert index to word or word to index"""

    def __init__(self, vocab):
        
        self.word_list = vocab
        self.word2idx_dict = {w:n_w for n_w, w in enumerate(self.word_list)}
        self.vocab_size = len(self.word_list)
        self.unk2idx = self.word2idx_dict['<unk>'] if '<unk>' in self.word2idx_dict else None

    def idx2word(self, n_w):

        return self.word_list[n_w]

    def word2idx(self, w):
        if w in self.word2idx_dict:
            return self.word2idx_dict[w]
        elif self.unk2idx is not None:
            return self.unk2idx
        else:
            raise ValueError(f'word {w} not in dictionary (while dictionary does not contain <unk>)')

    def tokenize_and_index(self, sentence):
        inds = [self.word2idx(w) for w in tokenize(sentence)]

        return inds

In [ ]:
class Data_Loaders:

    """Aim to convert index to word or word to index"""

    def __init__(self, data_dir, df_train, df_val, batch_size, img_size, normalised_box):
        self.img_size = img_size
        self.df_train = df_train
        self.df_val = df_val
        self.batch_size = batch_size
        self.question_vocab, self.max_question_len = self.make_vocab_questions(pd.concat([df_train["question"],df_val["question"]], axis = 0))
        self.question_vocab_size = len(self.question_vocab)
        self.question_dict = Text_Dict(self.question_vocab)
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.data_dir = data_dir
        self.normalised_box  = normalised_box
        os.system(f'mkdir {self.data_dir + "resized_imgs"}')
        os.system(f'mkdir {self.data_dir + "resized_boxes"}')
        os.system(f'mkdir {self.data_dir + "processed_boxes"}')
        os.system(f'saved_images')
        #self.transform = transforms.Compose([transforms.ToTensor(),
        #                                     transforms.Normalize((0.485, 0.456, 0.406),\
        #                                                          (0.229, 0.224, 0.225))])
     #Image and BBox Processing 
     #                                                             
    def create_mask(self, bb, x):
        """Creates a mask for the bounding box of same shape as image"""
        cols, rows,*_ = x.shape
        Y = np.zeros((cols, rows))
        bb = bb.astype(np.int64)
        Y[bb[1]:bb[3], bb[0]:bb[2]] = 1.
        return Y

    def mask_to_bb(self, Y):

        """Convert mask Y to a bounding box, assumes 0 as background nonzero object"""

        rows , cols = np.nonzero(Y)
        if len(cols)==0: 
            return np.zeros(4, dtype=np.float64)
        top_row = np.min(rows)
        left_col = np.min(cols)
        bottom_row = np.max(rows)
        right_col = np.max(cols)
        return np.array([left_col, top_row, right_col, bottom_row], dtype=np.int64)

    def xyxy2xywh(self, box):

        """Normalise box from xyXY -> xywh"""

        xmin, ymin, xmax, ymax = box
        w = xmax - xmin         # Width of the box
        h = ymax - ymin         # Height of the box
        x_center = (xmin + (w/2))/self.img_size 
        y_center = (ymin + (h/2))/self.img_size 
        new_w = w/self.img_size 
        new_h = h/self.img_size
        return np.array([x_center,y_center, new_w, new_h], dtype=np.float64)

    def resize_image_bb(self, img_path, bb, sz):

        """Resize an image and its bounding box and write image to new path"""
        img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
        mask = self.create_mask(bb, img)
        img_resized = cv2.resize(img, (sz, sz))
        mask_resized = cv2.resize(mask, (sz, sz))
        bb_resized = self.mask_to_bb(mask_resized)
        processed_box = self.xyxy2xywh(bb_resized)
        return img_resized, bb_resized, processed_box
    
    #Question Proxessing
    def make_vocab_questions(self, questions):

        """Make dictionary for questions and save them into text file."""

        vocab_set = set()
        SENTENCE_SPLIT_REGEX = re.compile(r'(\W+)')
        question_length = []
        set_question_length = [None]*len(questions)
        for iquestion, question in enumerate(questions):
            words = SENTENCE_SPLIT_REGEX.split(question.lower())
            words = [w.strip() for w in words if len(w.strip()) > 0]
            vocab_set.update(words)
            set_question_length[iquestion] = len(words)
        question_length += set_question_length

        vocab_list = list(vocab_set)
        vocab_list.sort()
        vocab_list.insert(0, '<pad>')
        vocab_list.insert(1, '<unk>')
        return vocab_list, np.max(question_length)

    #data batch processing
    def vqa_processing(self,df):
        """Create a data list to store all raw data simples"""
        data = []
        for idx in range(len(df)):
            image_path, _, _, l, t, r,b, question = df.iloc[idx]
            data.append({
                "image_path" : image_path,
                "text_question": question,
                "bbox": np.array([l, t, r,b], dtype =np.float64) #Xmin Ymin Xmax Ymax
            })
        return data

    def data_processing(self, data, question_dict, transform):
        images = []
        text_questions = []
        token_questions = []
        bboxes = []
        image_paths = []
        for sample in data:
            file_name = sample["image_path"].split('/')[-1][:-4]
            if exists(self.data_dir + "resized_imgs/" + file_name + '.npy'):
                resized_image = np.load(self.data_dir + "resized_imgs/" + file_name + '.npy')
                bb_resized = np.load(self.data_dir + "resized_boxes/" + file_name + '.npy')
                processed_bb = np.load(self.data_dir + "processed_boxes/" + file_name + '.npy')
            else:
                resized_image,  bb_resized , processed_bb = self.resize_image_bb(sample["image_path"], sample["bbox"], self.img_size)
                np.save(self.data_dir + "resized_imgs/" + file_name + '.npy', resized_image)
                np.save(self.data_dir + "resized_boxes/" + file_name + '.npy', bb_resized)
                np.save(self.data_dir + "processed_boxes/" + file_name + '.npy', processed_bb)
                
            token_question = tokenize(sample["text_question"])
            question_2_idx = np.array([question_dict.word2idx(w) for w in token_question], dtype = int)
            question_2_idx = torch.tensor(question_2_idx)
            images.append(transform(resized_image))
            text_questions.append(sample["text_question"])
            token_questions.append(question_2_idx)
            if self.normalised_box:
                bboxes.append(processed_bb)
            else:
                bboxes.append(bb_resized)
            image_paths.append(sample["image_path"])
            
        images = np.array([np.array(image) for image in images], dtype = np.float64)
        bboxes = np.array(bboxes, dtype = np.float64)
        token_questions = nn.utils.rnn.pad_sequence(token_questions, padding_value = 0, batch_first =True)
        return {
            "image":torch.tensor(images),
            "question": token_questions,
            "text_question": text_questions,
            "bbox": torch.tensor(bboxes),
            "image_path": image_paths
        }

    def train_loader(self):
        train_data = self.vqa_processing(self.df_train)
        train_loader = torch.utils.data.DataLoader(
            dataset=train_data,
            batch_size=self.batch_size,
            collate_fn=lambda x: self.data_processing(x, self.question_dict, self.transform))
        return train_loader

    def val_loader(self):
        train_data = self.vqa_processing(self.df_val)
        train_loader = torch.utils.data.DataLoader(
            dataset=train_data,
            batch_size=self.batch_size,
            collate_fn=lambda x: self.data_processing(x, self.question_dict, self.transform))
        return train_loader

# Model

In [ ]:
class ImgEncoder(nn.Module):

    def __init__(self, embedding_size):
        """(1) Load the pretrained model as you want.
               cf) one needs to check structure of model using 'print(model)'
                   to remove last fc layer from the model.
           (2) Replace final fc layer (score values from the ImageNet)
               with new fc layer (image feature).
           (3) Normalize feature vector.
        """
        super(ImgEncoder, self).__init__()
        self.cnn = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1').features
        self.fc = nn.Sequential(nn.Linear(self.cnn[-3].out_channels, embedding_size),
                                nn.Tanh())


    def forward(self, image):
        """Extract feature vector from image vector.
    #     """
        with torch.no_grad():
            img_feature = self.cnn(image)
        img_feature = img_feature.view(-1, 512, 20*20).transpose(1,2)
        img_feature = self.fc(img_feature)
        return img_feature

In [ ]:
class QstEncoder(nn.Module):

    def __init__(self, qst_vocab_size, word_embed_size, embed_size, num_layers, hidden_size):

        super(QstEncoder, self).__init__()
        self.word2vec = nn.Embedding(qst_vocab_size, word_embed_size)

        self.lstm = nn.LSTM(word_embed_size, hidden_size, batch_first=True)

    def forward(self, question):

        qst_vec = self.word2vec(question)                             # [batch_size, max_qst_length, word_embed_size]                            # [max_qst_length, batch_size, word_embed_size]
        _, (hidden, cell) = self.lstm(qst_vec)                        # [num_layers, batch_size, hidden_size]                   # [num_layers, batch_size, 2*hidden_size]
        return hidden[0]

In [ ]:
class Attention(nn.Module):
    def __init__(self, k, d, dropout=True):
        """Stacked attention Module
        """
        super(Attention, self).__init__()
        self.ff_image = nn.Linear(d, k)
        self.ff_questions = nn.Linear(d, k)
        self.dropout = nn.Dropout(p=0.5)
        self.ff_attention = nn.Linear(k, 1)

    def forward(self, vi, vq):
        """Extract feature vector from image vector.
        """
        # [Batch_size, Number of image regions, Representation Dimention] -> # [Batch_size, Number of image regions, k]
        hi = self.ff_image(vi)

        # [Batch_size, Representation Dimention] -> # [Batch_size, 1, k]
        hq = self.ff_questions(vq).unsqueeze(dim=1)

        # [Batch_size, Number of image regions,  k]
        ha = torch.tanh(hi+hq)
        if self.dropout:
            ha = self.dropout(ha)
        
        # Linear Attention
        # [Batch_size, Number of image regions,  k] -> [Batch_size, Number of image regions,  k]
        ha = self.ff_attention(ha)
        # [Batch_size, Number of image regions,  k] -> N * 196
        pi = torch.softmax(ha, dim=1)
        self.pi = pi
        vi_attended = (pi * vi).sum(dim=1)
        u = vi_attended + vq
        return u

In [ ]:
class VqaModel(nn.Module):

    def __init__(self,embedding_size, question_vocab_size, word_embedding_size, hidden_size, num_layers, num_att_layers):
        """
        Fusing Image feature and question feature using Full Connected Layer
        """
        super(VqaModel, self).__init__()
        self.image_encoder =  ImgEncoder(embedding_size)
        self.question_encoder = QstEncoder(question_vocab_size, word_embedding_size, embedding_size,num_layers, hidden_size)
        self.Stacked_Attend = nn.ModuleList(
            [Attention(k=512, d=embedding_size)] * num_att_layers)
        self.tanh = nn.Tanh()
        self.mlp = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(embedding_size, 4))

    def forward(self, image, question):
        img_feature = self.image_encoder(image)
        qst_feature = self.question_encoder(question)
        vi = img_feature
        u = qst_feature
        for attn_layer in self.Stacked_Attend:
            u = attn_layer(vi, u)
        combined_feature = self.mlp(u) 
        return combined_feature.sigmoid()

# Training

In [ ]:
def xywh_to_xyxy_batch(boxes1, boxes2, img_size):
    boxes1 = boxes1*img_size
    boxes2 = boxes2*img_size
    boxes1[:,0], boxes1[:,2] = boxes1[:, 0] - boxes1[:, 2] / 2, boxes1[:, 0] + boxes1[:, 2] / 2
    boxes1[:,1], boxes1[:,3] = boxes1[:, 1] - boxes1[:, 3] / 2, boxes1[:, 1] + boxes1[:, 3] / 2
    boxes2[:,0], boxes2[:,2] = boxes2[:, 0] - boxes2[:, 2] / 2, boxes2[:, 0] + boxes2[:, 2] / 2
    boxes2[:,1], boxes2[:,3] = boxes2[:, 1] - boxes2[:, 3] / 2, boxes2[:, 1] + boxes2[:, 3] / 2
    return boxes1, boxes2

In [ ]:
def bbox_iou(boxes1, boxes2, img_size, xyxy = False):
    """
    Returns the IoU of two bounding boxes
    """
    if not xyxy:
        boxes1, boxes2 = xywh_to_xyxy_batch(boxes1, boxes2, img_size)
    # Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = boxes1[:, 0], boxes1[:, 1], boxes1[:, 2], boxes1[:, 3]
    b2_x1, b2_y1, b2_x2, b2_y2 = boxes2[:, 0], boxes2[:, 1], boxes2[:, 2], boxes2[:, 3]

    # get the coordinates of the intersection rectangle
    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)
    # Intersection area
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1, 0) * torch.clamp(inter_rect_y2 - inter_rect_y1, 0)
    # Union Area
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    
    ious = inter_area / (b1_area + b2_area - inter_area + 1e-16)
    return ious
    

In [ ]:
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss(reduction='mean')
        
    def forward(self, pred, actual):
        loss = torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))
        return loss

In [ ]:
def save_prediction(img_path, pred_bb, gt_bb, question, epoch):
    img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    result = img.copy()
    cv2.rectangle(result, (int(gt_bb[0]), int(gt_bb[1])), (int(gt_bb[2]), int(gt_bb[3])), (0, 255, 0), 3)
    cv2.rectangle(result, (int(pred_bb[0]), int(pred_bb[1])), (int(pred_bb[2]), int(pred_bb[3])), (255, 0, 0), 3)
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title(question)
    plt.savefig(f"./saved_images/epoch_{epoch}_" + img_path.split("/")[-1])
    plt.close()

In [ ]:
def train_loop(epoch, data_loaders, model, loss_fn, optimizer, train_table):
    start_time = time.time()
    running_loss = 0.0
    running_iou = 0.0
    train_loader = data_loaders.train_loader()
    batch_size = data_loaders.batch_size
    size = len( data_loaders.df_train)
    num_batch =  size/batch_size
    for batch_idx, batch_data in enumerate(train_loader):
        images, questions = batch_data["image"].float().to(DEVICE), batch_data["question"].to(DEVICE)
        targets = batch_data["bbox"].float().to(DEVICE)
        
        # Compute prediction and loss
        preds = model(images, questions)
        loss = loss_fn(preds, targets)
        running_loss +=  loss.item()

        #Cal ious
        if data_loaders.normalised_box:
            ious = bbox_iou(preds, targets, data_loaders.img_size)
        else:
            ious = bbox_iou(preds, targets, data_loaders.img_size, xyxy=True)
        mean_iou = torch.mean(ious).item()

        running_iou += mean_iou
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #WandB log
        if LOG_WANDB:
            wandb.log({
                "loss_step": loss.item(),
                "iou_step": mean_iou
            })
        if (batch_idx +1) % 200 == 0:
            print(f'    [{(batch_idx+1)*batch_size}/{size}] | Loss : {loss.item():>7f} | IoU: {mean_iou:>5f}')
    running_loss = running_loss/num_batch
    running_iou = running_iou/num_batch

    #WandB log
    if LOG_WANDB:
        wandb.log({
            "epoch": epoch+1,
            "loss": running_loss,
            "iou": running_iou
        })
        if (epoch + 1) % SAVE_IMAGE_FREQ == 0:
            if data_loaders.normalised_box:
                preds, targets = xywh_to_xyxy_batch(preds, targets, data_loaders.img_size)
            for idx in range(len(targets)):
                img_path = batch_data['image_path'][idx]
                ques_text = batch_data['text_question'][idx]
                target = targets[idx]
                prediction = preds[idx]
                save_prediction(img_path, prediction, target, ques_text, epoch+1)
                train_table.add_data(epoch+1, "train", wandb.Image(f"./saved_images/epoch_{epoch+1}_" + img_path.split("/")[-1]), ious[idx].item())
            
    print(f'\nTRAINING | Loss: {running_loss:>7f} | IoU: {running_iou:>5f} | Running_time : {time.time() - start_time:>5f}s')
    return running_loss, running_iou, train_table

In [ ]:
def val_loop(epoch, data_loaders, model, loss_fn, val_table):
    start_time = time.time()
    running_loss = 0.0
    running_iou = 0.0
    val_loader = data_loaders.val_loader()
    batch_size = data_loaders.batch_size
    num_batch =  len( data_loaders.df_val)/batch_size
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(val_loader):
            images, questions = batch_data["image"].float().to(DEVICE), batch_data["question"].to(DEVICE)
            targets = batch_data["bbox"].float().to(DEVICE)
            # Compute prediction and loss
            preds = model(images, questions)
            loss = loss_fn(preds, targets)
            running_loss +=  loss.item()

            #Cal ious
            if data_loaders.normalised_box:
                ious = bbox_iou(preds, targets, data_loaders.img_size)
            else:
                ious = bbox_iou(preds, targets, data_loaders.img_size, xyxy=True)
                
            running_iou += torch.mean(ious).item()
            
    running_loss = running_loss/num_batch
    running_iou = running_iou/num_batch
    print(f'VAL      | Loss: {running_loss:>7f} | IoU: {running_iou:>5f} | Running_time : {time.time() - start_time:>5f}s')

    if LOG_WANDB:
        wandb.log({
            "epoch": epoch+1,
            "val_loss":  running_loss,
            "val_iou": running_iou
        })
        if (epoch + 1) % SAVE_IMAGE_FREQ == 0:
            if data_loaders.normalised_box:
                preds, targets = xywh_to_xyxy_batch(preds, targets, data_loaders.img_size)
            for idx in range(len(targets)):
                img_path = batch_data['image_path'][idx]
                ques_text = batch_data['text_question'][idx]
                target = targets[idx]
                prediction = preds[idx]
                save_prediction(img_path, prediction, target, ques_text, epoch+1)
                val_table.add_data(epoch+1, "val", wandb.Image(f"./saved_images/epoch_{epoch+1}_" + img_path.split("/")[-1]), ious[idx].item())
    return running_loss, running_iou, val_table

In [ ]:

    
data_loaders = Data_Loaders(DATA_DIR, df_train, df_val, BATCH_SIZE, IMAGE_SIZE, normalised_box=True)

print('create the model')
model = VqaModel(
    embedding_size = EMBEDDIND_SIZE,
    question_vocab_size = data_loaders.question_vocab_size,
    word_embedding_size = WORD_EMBEDING_SIZE,
    hidden_size=LSTM_HIDDEN_SIZE,
    num_layers=NUMBER_LSTM_LAYER,
    num_att_layers=3 
).to(DEVICE)
 
params = model.parameters()

optimizer = torch.optim.Adam(params, lr=LEARNING_RATE)
loss_fn = RMSLELoss()  #or nn.MSELoss(reduction='mean')

max_iou = 0
if LOG_WANDB:
    os.system('mkdir saved_images')
    columns=["epoch", "dataset", "image", "iou"]
    train_table = wandb.Table(columns=columns)
    val_table = wandb.Table(columns=columns)
else:
    train_table = None
    val_table = None
logs = {
        "loss": [],
        "iou": [],
        "val_loss": [],
        "val_iou": []
        }

for epoch in range(NUM_EPOCHS):
    print("------------------------------------------------------------------------------------------------")
    print('------------------------------------------------------------------------------------------------')
    print(f"EPOCH {epoch+1}\n")

    loss, iou, train_table = train_loop(epoch, data_loaders, model, loss_fn, optimizer, train_table)
    val_loss, val_iou, val_table= val_loop(epoch, data_loaders, model, loss_fn, val_table)
    if val_iou > max_iou:
        max_iou = val_iou
        torch.save(model, 'saved_models/model.pt')
    
    logs["loss"].append(loss)
    logs["iou"].append(iou)
    logs["val_loss"].append(val_loss)
    logs["val_iou"].append(val_iou)

In [ ]:
plt.figure(figsize=(15,7))
plt.subplot(1, 2, 1)
plt.plot(logs["loss"], label='train')
plt.plot(logs["val_loss"], label='val')
plt.title("Loss")
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('RMSLE')

plt.subplot(1, 2, 2) # index 2
plt.plot(logs["iou"], label='train')
plt.plot(logs["val_iou"], label='val')
plt.title("IoU")
plt.xlabel('Epoch')
plt.legend()
plt.grid()
plt.ylabel('IoU')
plt.show()

In [ ]:
if LOG_WANDB:
    wandb.log({"train example": train_table})
    wandb.log({"val example": val_table})
    os.system('rm -rf saved_images')
    wandb.finish()